<table>
  <tr>
      <td><div align="left"><font size="20" >Finding Aruco markers</font></div></td>
     <td><img src="images/RVSS-logo.png" width="400"></td>
  </tr>
</table>

In [ ]:
%matplotlib notebook

import sys
!{sys.executable} -m pip install machinevision-toolbox-python # used to install machinevision-toolbox into the kernel
!{sys.executable} -m pip install opencv-contrib-python==4.4.0.46 # update opencv to 4.4.0

import numpy as np
import math
from spatialmath import SE3, SO3
from spatialmath.base import e2h, h2e
from machinevisiontoolbox import Image, plot_point
import cv2

Aruco markers were developed as a simple way to mark things for augmented reality (AR) applications, hence the AR in their name.  They are just high-contrast grids that encode a number as well as position and orientation.

We will load a synthetic image of some Aruco markers, two with distortions, onto a picture from western Queensland.

In [ ]:
scene = Image('images/aruco.png')
scene.disp();

Before we can use OpenCV's Aruco module we need to define a dictionary that describes the Aruco marker pattern.  There are many different Aruco marker families, here we are using `4x4_1000` which is a $4 \times 4$ grid of squares that can encode numbers from 0 to 9999.

In [ ]:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_1000)

We pass the dictionary and the image to the marker detector function

In [ ]:
markers, ids, _ = cv2.aruco.detectMarkers(scene.image, dictionary)

`ids` is an array containing the identities of the three markers found in this image

In [ ]:
ids

`markers` contains a list of $4\times 2$ arrays with the estimated corner locations for each marker

In [ ]:
markers

We can iterate over the markers and overlay the markers and Aruco id on the original image

In [ ]:
for id, corners in zip(ids, markers):
    plot_point(corners.T, 'bs')
    centre = np.mean(corners, axis=1)
    plot_point(tuple(centre.flatten()), 'yo', text=f"{id[0]}", color='yellow')


For each marker we have the coordinates of four corners which is enough to estimate the orientation of the planar marker in 3D space.  In order to do this we need to know some parameters of the imaging setup

In [ ]:
f = 500  # focal length of the camera in units of pixels (wild guess)
side = 1  # the side length of the marker in units of metres

From this we can estimate the camera's intrinsic parameter (lesson A2) matrix K

In [ ]:
K = np.array([[f, 0, 344], [0, f, 260], [0, 0, 1]])
K

and then estimate the position and orientation of each marker

In [ ]:
rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(markers, side, K, None)

`rvecs` is the orientation of each marker expressed as an Euler vector

In [ ]:
rvecs

We can convert the Euler vector to roll-pitch-yaw angles in units of degrees

In [ ]:
for rvec in rvecs:
    R = SO3.EulerVec(rvec.flatten())
    print(R.rpy(unit='deg'))

`tvecs` is the 3D position of the centre of the marker with respect to the camera

In [ ]:
tvecs